<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [67]:
import mediapipe as mp
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import gc
from operator import add
tqdm.pandas()

In [21]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [22]:
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=1) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
            
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [23]:
results

mediapipe.python.solution_base.SolutionOutputs

In [24]:
len(results.pose_landmarks.landmark)

33

In [25]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [26]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [27]:
def extract_keypoints(results):
#    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [28]:
result_test = extract_keypoints(results)

In [29]:
result_test

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [30]:
def signvideodataframe(filepath):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath)

    # Define the path to the videos directory
    path = 'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/'

    # Create a 'Path' column by concatenating the path with 'Video file' column
    df['Path'] = path + df['Video file']
    
#    df['Gloss'] = df['Gloss'].str.replace('\d+', '',regex=True)

    # Calculate the frequency of each gloss and create a 'frequency' column
    df['Frequency'] = df['Gloss'].map(df['Gloss'].value_counts())

    # Sort the DataFrame by the 'Gloss' column
    df = df.sort_values(by='Gloss')

    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)

    return df

In [31]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [36]:
traindf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/train.csv')

In [37]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P31,3827306090663467-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P37,16792698524451422-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P11,6868778695018762-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P11,6870709051348651-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P50,0719792557216079-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [38]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
40149,P37,9716493262876276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40150,P31,7550572181460327-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40151,P46,47985881750082227-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40152,P50,04671245574824856-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40153,P51,19959052532136146-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [39]:
traindf['Path'][0]

'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/3827306090663467-1 DOLLAR.mp4'

In [40]:
cap = cv2.VideoCapture(traindf['Path'][0])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Posture
        mp_drawing.draw_landmarks(
            frame, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [41]:
traindf.sample(20)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
27666,P33,38253788313087966-RACE.mp4,RACE,F_02_075,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
28666,P37,8020060049631272-RIGHT 2.mp4,RIGHT2,C_01_070,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
9271,P40,5336635865778496-DICE.mp4,DICE,D_02_036,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
30435,P16,08013205314920557-SHAME ON_YOU.mp4,SHAMEONYOU,H_01_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
20590,P7,34292735943128716-LOCKER.mp4,LOCKER,K_01_126,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
30006,P43,17195855615886702-SCULPT.mp4,SCULPT,J_01_028,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
34604,P3,17146468242733048-SWEET.mp4,SWEET,J_03_029,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
39486,P40,7278194454734639-WOMAN 2.mp4,WOMAN2,K_01_101,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
28099,P40,45570209958510843-REGISTER.mp4,REGISTER,F_01_021,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
26878,P27,9525196309623671-PREFER.mp4,PREFER,B_01_078,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [43]:
valdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/val.csv') 
testdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/test.csv')

In [47]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P26,22595012150860327-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
1,P39,7421622940519235-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
2,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
3,P12,9219095671540121-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P21,1448188216215387-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [48]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P18,23521769221811684-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
1,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
2,P49,4893817008748198-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
3,P17,13991818149960333-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
4,P49,34625615110480457-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [56]:
traindf = pd.concat([traindf,valdf,testdf])

In [57]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
32936,P18,4320702510886756-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32937,P9,7676354653247301-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32938,P47,5386272465310649-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32939,P18,738440364224181-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32940,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [58]:
# Calculate the frequency of each gloss and create a 'frequency' column
traindf['Frequency'] = traindf['Gloss'].map(traindf['Gloss'].value_counts())

# Sort the DataFrame by the 'Gloss' column
traindf = traindf.sort_values(by='Gloss')

# Reset the index of the DataFrame
traindf = traindf.reset_index(drop=True)

In [59]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P31,3827306090663467-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31
1,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31
2,P18,23521769221811684-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31
3,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31
4,P49,4893817008748198-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31


In [60]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
83394,P40,29614325657927165-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32
83395,P33,38572260119226276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32
83396,P16,29179019672445805-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32
83397,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32
83398,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32


In [61]:
traindf.describe()

,Frequency
count,83399.000000
mean,30.635631
std,1.741494
min,21.000000
25%,30.000000
50%,31.000000
75%,31.000000
max,45.000000


In [77]:
tqdm.pandas()
traindf[['Frames', 'FPS', 'Length']] = traindf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████| 83399/83399 [51:17<00:00, 27.10it/s]


In [79]:
#traindf.to_csv('//home/kristian/ASL_Citizen/splits/all_videos.csv')

In [32]:
traindf=pd.read_csv('//home/kristian/ASL_Citizen/splits/all_videos.csv',index_col=0)

In [33]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
83394,P40,29614325657927165-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,103,30,3
83395,P33,38572260119226276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,131,29,5
83396,P16,29179019672445805-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,57,30,2
83397,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,53,30,2
83398,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,50,30,2


In [34]:
traindf.describe()

,Frequency,Frames,FPS,Length
count,83399.000000,83399.000000,83399.000000,83399.000000
mean,30.635631,82.755585,29.146357,2.849231
std,1.741494,37.687446,2.767382,1.268591
min,21.000000,3.000000,11.000000,0.000000
25%,30.000000,58.000000,29.000000,2.000000
50%,31.000000,75.000000,30.000000,3.000000
75%,31.000000,96.000000,30.000000,3.000000
max,45.000000,680.000000,120.000000,23.000000


In [35]:
traindf2 = traindf[(traindf['Length'] > 0) & (traindf['FPS'] > 28) &(traindf['Frames'] < 61) ]
traindf2 = traindf2.reset_index(drop=True)

In [36]:
traindf2.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31,57,29,2
1,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31,45,30,2
2,P17,13991818149960333-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31,56,30,2
3,P6,641444141964802-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31,49,29,2
4,P22,86640342214128-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,31,50,29,2


In [37]:
traindf2.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
21320,P27,2780771420581618-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,47,30,2
21321,P11,6893191909065832-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,50,29,2
21322,P16,29179019672445805-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,57,30,2
21323,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,53,30,2
21324,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,32,50,30,2


In [38]:
traindf2.describe()

,Frequency,Frames,FPS,Length
count,21325.000000,21325.000000,21325.000000,21325.000000
mean,30.703400,50.040469,29.689004,1.795545
std,1.747375,7.409050,0.525716,0.403312
min,21.000000,16.000000,29.000000,1.000000
25%,30.000000,46.000000,29.000000,2.000000
50%,31.000000,51.000000,30.000000,2.000000
75%,31.000000,56.000000,30.000000,2.000000
max,45.000000,60.000000,60.000000,2.000000


In [39]:
# Calculate the frequency of each gloss and create a 'frequency' column
traindf2['Frequency'] = traindf2['Gloss'].map(traindf2['Gloss'].value_counts())

    # Sort the DataFrame by the 'Gloss' column
traindf2 = traindf2.sort_values(by='Gloss')

# Reset the index of the DataFrame
traindf2 = traindf2.reset_index(drop=True)

In [40]:
traindf2.describe()

,Frequency,Frames,FPS,Length
count,21325.000000,21325.000000,21325.000000,21325.000000
mean,9.043236,50.040469,29.689004,1.795545
std,2.965475,7.409050,0.525716,0.403312
min,1.000000,16.000000,29.000000,1.000000
25%,7.000000,46.000000,29.000000,2.000000
50%,9.000000,51.000000,30.000000,2.000000
75%,11.000000,56.000000,30.000000,2.000000
max,19.000000,60.000000,60.000000,2.000000


In [41]:
traindf2 = traindf2[(traindf2['Frequency'] > 14) ]
    # Sort the DataFrame by the 'Gloss' column
traindf2 = traindf2.sort_values(by='Gloss')

# Reset the index of the DataFrame
traindf2 = traindf2.reset_index(drop=True)

In [42]:
traindf2.describe()

,Frequency,Frames,FPS,Length
count,920.000000,920.000000,920.000000,920.000000
mean,15.641304,48.479348,29.695652,1.716304
std,0.904366,8.424123,0.498946,0.451036
min,15.000000,16.000000,29.000000,1.000000
25%,15.000000,44.000000,29.000000,1.000000
50%,15.000000,50.000000,30.000000,2.000000
75%,16.000000,55.000000,30.000000,2.000000
max,19.000000,60.000000,31.000000,2.000000


In [43]:
#traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)

In [44]:
traindf2['Gloss'].nunique()

59

In [45]:
words = traindf2['Gloss'].unique()
words

array(['AND', 'ASK', 'BEST', 'BETTER', 'BLOND', 'BOLT', 'CANDY2', 'CLICK',
       'COME', 'DEAF1', 'DENTIST2', 'DOG1', 'EAST', 'EAT1', 'ELEVEN',
       'FAKE', 'FIND', 'FINE1', 'FOND', 'FOUR', 'GALLAUDET', 'GAY',
       'GIVE', 'GOTCHA', 'GREEN', 'HANDSOME3', 'HONEY1', 'HOT', 'HOW2',
       'HURDLE/TRIP1', 'LIGHTER', 'LOOKAT', 'LUCKY', 'NINE', 'NORTH',
       'NOTCARE', 'OFF', 'OK', 'PATIENT2', 'PINK', 'REALLY', 'RECENT1',
       'SEEM', 'SELECT', 'SEND', 'SHORTDISTANCE', 'SIX', 'SPITOUT',
       'STRAIGHT', 'STRANGE', 'TAKE', 'TAKEPILL', 'THINK', 'THIRD1',
       'TINY', 'TWINS1', 'UNDERSTAND', 'WINK', 'WISH'], dtype=object)

In [46]:
traindf2['Gloss'] = traindf2['Gloss'].str.replace('\d+', '',regex=True)

In [47]:
words = traindf2['Gloss'].unique()
words

array(['AND', 'ASK', 'BEST', 'BETTER', 'BLOND', 'BOLT', 'CANDY', 'CLICK',
       'COME', 'DEAF', 'DENTIST', 'DOG', 'EAST', 'EAT', 'ELEVEN', 'FAKE',
       'FIND', 'FINE', 'FOND', 'FOUR', 'GALLAUDET', 'GAY', 'GIVE',
       'GOTCHA', 'GREEN', 'HANDSOME', 'HONEY', 'HOT', 'HOW',
       'HURDLE/TRIP', 'LIGHTER', 'LOOKAT', 'LUCKY', 'NINE', 'NORTH',
       'NOTCARE', 'OFF', 'OK', 'PATIENT', 'PINK', 'REALLY', 'RECENT',
       'SEEM', 'SELECT', 'SEND', 'SHORTDISTANCE', 'SIX', 'SPITOUT',
       'STRAIGHT', 'STRANGE', 'TAKE', 'TAKEPILL', 'THINK', 'THIRD',
       'TINY', 'TWINS', 'UNDERSTAND', 'WINK', 'WISH'], dtype=object)

In [48]:
traindf=traindf2

In [52]:
cap = cv2.VideoCapture(traindf.Path[1])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=2) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        # Mano izquieda (azul)
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Mano derecha (verde)
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [53]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=1,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [54]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file_flip(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=1,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.flip(frame, 1)
        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [51]:
traindf['Path'] = traindf['Path'].str.replace('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/', 'file:///home/kristian/ASL_Citizen/',regex=True)

In [55]:
extract_keypoints_from_file(traindf.Path[0]).shape

(43, 126)

In [56]:
cap = cv2.VideoCapture(traindf.Path[0])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

43


In [57]:
type(extract_keypoints_from_file(traindf.Path[643]))

numpy.ndarray

In [156]:
import os
import shutil

# Source Folder
source_folder = '/media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/'

# Destination FOlder
destination_folder = '/home/kristian/ASL_Citizen/videos/'

# Loop through each file in the source folder
for filename in os.listdir(source_folder):
    if filename in traindf['Video file'][:].values:
        source_path = os.path.join(source_folder, filename)
        destination_path = os.path.join(destination_folder, filename)
        shutil.copy(source_path, destination_path)
        print(f"File '{filename}' copied.")

print("Copying complete.")

File '6843816941433676-HURDLE-TRIP.mp4' copied.
File '5270060903242877-DENTIST 2.mp4' copied.
File '09014404715027236-WINK.mp4' copied.
File '34178569625029054-TWINS.mp4' copied.
File '18352529322609779-BEST.mp4' copied.
File '8421220383934189-HOT.mp4' copied.
File '8138364046329691-FOUR.mp4' copied.
File '5141622294161379-FOUR.mp4' copied.
File '22096800930230853-TAKE PILL.mp4' copied.
File '4524158338314277-EAT.mp4' copied.
File '45242458406215746-TAKE PILL.mp4' copied.
File '10685950953531731-THIRD.mp4' copied.
File '5745729001649573-OK.mp4' copied.
File '5747517326015636-SELECT.mp4' copied.
File '5575086704295376-CLICK.mp4' copied.
File '10254755835642815-LUCKY.mp4' copied.
File '5976950094284144-ASK.mp4' copied.
File '42041801095735476-CANDY 2.mp4' copied.
File '9331308031305054-CANDY 2.mp4' copied.
File '4728590611874972-STRAIGHT.mp4' copied.
File '5308291392983451-RECENT.mp4' copied.
File '5309189359554582-HANDSOME 4.mp4' copied.
File '1340015962713661-PINK.mp4' copied.
File '54

File '2980712905804166-SELECT.mp4' copied.
File '4788883809061111-THIRD.mp4' copied.
File '6698348047100313-NORTH.mp4' copied.
File '300554219176699-SEND.mp4' copied.
File '5607838772101854-TWINS.mp4' copied.
File '30539347753669954-GREEN.mp4' copied.
File '5739215722518052-EAT.mp4' copied.
File '14269575017191238-EAST.mp4' copied.
File '8864496316656505-LIGHTER.mp4' copied.
File '2495443890240825-GIVE.mp4' copied.
File '756000062080056-GOTCHA.mp4' copied.
File '7560560007286001-FOND.mp4' copied.
File '7613143471093098-LIGHTER.mp4' copied.
File '8569232579936605-THINK.mp4' copied.
File '8569393046306124-CLICK.mp4' copied.
File '07478302952605298-GAY.mp4' copied.
File '8193368982950104-HOW 2.mp4' copied.
File '8193754472141177-HONEY 1.mp4' copied.
File '43107482372128714-REALLY.mp4' copied.
File '6951725928781516-WISH.mp4' copied.
File '006951411960890663-FINE.mp4' copied.
File '919443966194881-EAST.mp4' copied.
File '8440442611677397-FOUR.mp4' copied.
File '5370118594616453-STRANGE.mp4

File '9742883685495938-DEAF.mp4' copied.
File '03845734114791988-BLOND.mp4' copied.
File '4611032306786558-TWINS.mp4' copied.
File '0020382808137153763-ELEVEN.mp4' copied.
File '5170125336404756-COME.mp4' copied.
File '03150905619350208-SELECT.mp4' copied.
File '8122344601646359-LOOK AT.mp4' copied.
File '8123288036507608-GAY.mp4' copied.
File '10827398525400311-OFF.mp4' copied.
File '5650868043704504-NINE.mp4' copied.
File '8003732238675179-DEAF.mp4' copied.
File '3981207314157731-AND.mp4' copied.
File '08478560862167717-RECENT.mp4' copied.
File '08497040540501066-SHORT DISTANCE.mp4' copied.
File '043016679537065894-FOND.mp4' copied.
File '4239903470632247-BOLT.mp4' copied.
File '4487899898775074-LIGHTER.mp4' copied.
File '6504234134902995-UNDERSTAND.mp4' copied.
File '6156241027736771-RECENT.mp4' copied.
File '42978197247437233-OFF.mp4' copied.
File '8285963105812504-TAKE.mp4' copied.
File '19543181290734046-CANDY 2.mp4' copied.
File '2850100795370789-SIX.mp4' copied.
File '970502136

File '43905250683951924-STRANGE.mp4' copied.
File '4576928379969838-EAST.mp4' copied.
File '2490150449515618-ASK.mp4' copied.
File '24921588003043138-BLOND.mp4' copied.
File '19022708495593288-SEEM.mp4' copied.
File '26319942987799516-BOLT.mp4' copied.
File '10888540499689392-HANDSOME 4.mp4' copied.
File '2274427695208907-LOOK AT.mp4' copied.
File '2274519559281516-HOW 2.mp4' copied.
File '8308912697239315-GIVE.mp4' copied.
File '830943775589045-LOOK AT.mp4' copied.
File '44964320032500793-WINK.mp4' copied.
File '09953186633346167-OFF.mp4' copied.
File '5268295753822807-STRANGE.mp4' copied.
File '3821027760511011-SEEM.mp4' copied.
File '20179596706480196-GAY.mp4' copied.
File '4626224421615077-CANDY 2.mp4' copied.
File '6491857817064066-PATIENT 2.mp4' copied.
File '15587296563996822-GREEN.mp4' copied.
File '9071310344664434-NORTH.mp4' copied.
File '4550932054745036-PINK.mp4' copied.
File '7660361464563388-GREEN.mp4' copied.
File '2792895407119611-GREEN.mp4' copied.
File '27946341175774

File '8618961603650133-FIND.mp4' copied.
File '06996746044765945-ELEVEN.mp4' copied.
File '6745834273988278-UNDERSTAND.mp4' copied.
File '25866202861571086-HOW 2.mp4' copied.
File '13734587041918878-BOLT.mp4' copied.
File '9864209236776549-DENTIST 2.mp4' copied.
File '7025796673937177-OFF.mp4' copied.
File '7072821167111014-ASK.mp4' copied.
File '3200787903337785-HOT.mp4' copied.
File '7769434916738955-EAT.mp4' copied.
File '5547021297445855-NINE.mp4' copied.
File '8598719617773274-TWINS.mp4' copied.
File '5240410951548111-GALLAUDET.mp4' copied.
File '5240887216596288-FIND.mp4' copied.
File '6876614460019714-LUCKY.mp4' copied.
File '7162964278884962-CLICK.mp4' copied.
File '2825233660609592-TWINS.mp4' copied.
File '5781025522650556-PATIENT 2.mp4' copied.
File '38426637881924686-FOND.mp4' copied.
File '8071758688201891-UNDERSTAND.mp4' copied.
File '6370702900476413-ASK.mp4' copied.
File '4163757443011573-SIX.mp4' copied.
File '6516460654825258-ELEVEN.mp4' copied.
File '83004477178377-BL

In [58]:
traindf.count()

Participant ID    920
Video file        920
Gloss             920
ASL-LEX Code      920
Path              920
Frequency         920
Frames            920
FPS               920
Length            920
dtype: int64

In [59]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [60]:
frames_from_file(traindf.Path[0])

(43, 30, 1)

In [64]:
traindf['Frames'].max()

60

In [ ]:
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60_hands/train_keypoints.npy", keypoints_series)
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file_flip)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60_hands/train_keypoints_flip.npy", keypoints_series)

 10%|████▏                                     | 92/920 [04:31<45:54,  3.33s/it]

In [66]:
tqdm.pandas()
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file_flip)

  0%|▏                                          | 4/920 [00:08<32:19,  2.12s/it]


KeyboardInterrupt: 

In [234]:
keypoints_series.shape

(920,)

In [231]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60/train_keypoints.npy", keypoints_series)

In [233]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60/train_keypoints.npy",allow_pickle=True)

In [235]:
max(keypoints.shape[0] for keypoints in keypoints_series)

60

In [236]:
min(keypoints.shape[0] for keypoints in keypoints_series)


16

In [237]:
max_len = traindf['Frames'].max()

In [238]:
max_len

60

In [239]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [240]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [241]:
keypoints_np.shape

(920, 60, 1662)

In [242]:
print(keypoints_np[6])

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.49567455  0.30508247 -1.63521326 ...  0.          0.
   0.        ]
 [ 0.495891    0.30340993 -1.66837811 ...  0.          0.
   0.        ]
 [ 0.49603844  0.30246556 -1.66233063 ...  0.          0.
   0.        ]]


In [243]:
keypoints_np.shape

(920, 60, 1662)

In [244]:
keypoints_np

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.48855895,  0.40100691, -0.92158365, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.48833609,  0.40101019, -0.91984463, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.48820359,  0.40102521, -0.91442269, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.48129115,  0.5108977 , -1.09578931, ...,  

In [245]:
keypoints_np.shape

(920, 60, 1662)

In [246]:
keypoints_np.shape

(920, 60, 1662)

# Preprocessing

In [1]:
import numpy as np
train_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60/train_keypoints.npy",allow_pickle=True)
#val_keypoints =np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy",allow_pickle=True)
#test_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy",allow_pickle=True)

print('Training Keypoints Shape:', train_keypoints.shape)
#print('Validation Kepoints Shape:', val_keypoints.shape)
#print('Test Keypoints Shape:',test_keypoints.shape)

Training Keypoints Shape: (920,)


In [2]:
max_len=60

In [3]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(train_keypoints), max_len, 1662))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(train_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
train_keypoints = keypoints_np
print('Training Keypoints Shape:', train_keypoints.shape)

Training Keypoints Shape: (920, 60, 1662)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import to_categorical
import numpy as np
from keras.preprocessing.sequence import pad_sequences

2023-08-13 22:57:56.212257: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-13 22:57:56.843911: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [45]:
traindf.head()

NameError: name 'traindf' is not defined

In [253]:
encoder = OrdinalEncoder()

In [254]:
traindf['Cat_label'] = encoder.fit_transform(traindf[['Gloss']]).astype(int)
traindf = traindf.sort_values(by='Gloss',)

In [255]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P29,6211759212759429-AND.mp4,AND,B_03_072,file:///home/kristian/ASL_Citizen/videos/62117...,15,43,30,1,0
14,P21,5038398188714364-AND.mp4,AND,B_03_072,file:///home/kristian/ASL_Citizen/videos/50383...,15,44,29,2,0
13,P16,3741021155965083-AND.mp4,AND,B_03_072,file:///home/kristian/ASL_Citizen/videos/37410...,15,56,29,2,0
12,P50,6610091794646322-AND.mp4,AND,B_03_072,file:///home/kristian/ASL_Citizen/videos/66100...,15,60,30,2,0
11,P49,7841237666552301-AND.mp4,AND,B_03_072,file:///home/kristian/ASL_Citizen/videos/78412...,15,41,30,1,0


In [256]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
906,P27,7262581620088506-WISH.mp4,WISH,C_01_053,file:///home/kristian/ASL_Citizen/videos/72625...,15,50,30,2,58
905,P7,9899925230812674-WISH.mp4,WISH,C_01_053,file:///home/kristian/ASL_Citizen/videos/98999...,15,60,30,2,58
918,P11,5986296474103647-WISH.mp4,WISH,C_01_053,file:///home/kristian/ASL_Citizen/videos/59862...,15,53,29,2,58
911,P35,3446645273513582-WISH.mp4,WISH,C_01_053,file:///home/kristian/ASL_Citizen/videos/34466...,15,51,29,2,58
919,P17,3815358216512903-WISH.mp4,WISH,C_01_053,file:///home/kristian/ASL_Citizen/videos/38153...,15,56,29,2,58


In [258]:
label_map = {label:num for num, label in enumerate(traindf['Gloss'].unique())}

In [259]:
label_map

{'AND': 0,
 'ASK': 1,
 'BEST': 2,
 'BETTER': 3,
 'BLOND': 4,
 'BOLT': 5,
 'CANDY': 6,
 'CLICK': 7,
 'COME': 8,
 'DEAF': 9,
 'DENTIST': 10,
 'DOG': 11,
 'EAST': 12,
 'EAT': 13,
 'ELEVEN': 14,
 'FAKE': 15,
 'FIND': 16,
 'FINE': 17,
 'FOND': 18,
 'FOUR': 19,
 'GALLAUDET': 20,
 'GAY': 21,
 'GIVE': 22,
 'GOTCHA': 23,
 'GREEN': 24,
 'HANDSOME': 25,
 'HONEY': 26,
 'HOT': 27,
 'HOW': 28,
 'HURDLE/TRIP': 29,
 'LIGHTER': 30,
 'LOOKAT': 31,
 'LUCKY': 32,
 'NINE': 33,
 'NORTH': 34,
 'NOTCARE': 35,
 'OFF': 36,
 'OK': 37,
 'PATIENT': 38,
 'PINK': 39,
 'REALLY': 40,
 'RECENT': 41,
 'SEEM': 42,
 'SELECT': 43,
 'SEND': 44,
 'SHORTDISTANCE': 45,
 'SIX': 46,
 'SPITOUT': 47,
 'STRAIGHT': 48,
 'STRANGE': 49,
 'TAKE': 50,
 'TAKEPILL': 51,
 'THINK': 52,
 'THIRD': 53,
 'TINY': 54,
 'TWINS': 55,
 'UNDERSTAND': 56,
 'WINK': 57,
 'WISH': 58}

In [260]:
traindf.sample(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
534,P36,48027094398019576-NORTH.mp4,NORTH,C_01_080,file:///home/kristian/ASL_Citizen/videos/48027...,15,59,30,2,34
783,P22,5617280048659572-TAKE.mp4,TAKE,G_01_093,file:///home/kristian/ASL_Citizen/videos/56172...,16,45,30,2,50
916,P6,9915177398162693-WISH.mp4,WISH,C_01_053,file:///home/kristian/ASL_Citizen/videos/99151...,15,32,30,1,58
328,P21,010356460960846414-GALLAUDET.mp4,GALLAUDET,B_02_010,file:///home/kristian/ASL_Citizen/videos/01035...,15,46,30,2,20
672,P42,37942641665406507-SEEM.mp4,SEEM,F_01_043,file:///home/kristian/ASL_Citizen/videos/37942...,16,55,30,2,42
10,P6,3981207314157731-AND.mp4,AND,B_03_072,file:///home/kristian/ASL_Citizen/videos/39812...,15,32,29,1,0
568,P14,8676867126730325-OFF.mp4,OFF,E_01_027,file:///home/kristian/ASL_Citizen/videos/86768...,17,52,29,2,36
544,P17,5281867130394469-NORTH.mp4,NORTH,C_01_080,file:///home/kristian/ASL_Citizen/videos/52818...,15,46,30,2,34
40,P50,5002484283202087-BEST.mp4,BEST,F_02_062,file:///home/kristian/ASL_Citizen/videos/50024...,16,60,30,2,2
104,P11,2987380177072183-CANDY 2.mp4,CANDY,D_02_079,file:///home/kristian/ASL_Citizen/videos/29873...,16,46,29,2,6


In [261]:
Ytrain = encoder.fit_transform(traindf[['Gloss']])
#Ytest = encoder.fit_transform(testdf[['Gloss']])
#Yval = encoder.fit_transform(valdf[['Gloss']])

In [262]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60/train_labels_array_1D.npy", Ytrain)
#np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_labels_array_1D.npy", Ytest)
#np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_labels_array_1D.npy", Yval)

In [5]:
Ytrain=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe-60/train_labels_array_1D.npy")
#Ytest=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_labels_array_1D.npy")
#Yval=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_labels_array_1D.npy")


In [6]:
Xtrain = train_keypoints
#Xtest = test_keypoints
#Xval = val_keypoints

In [7]:
n_classes = 59
print("Shape before one-hot encoding: ", Ytrain.shape)
Ytrain = to_categorical(Ytrain, n_classes)
print("Shape after one-hot encoding: ", Ytrain.shape)
#print("Shape before one-hot encoding: ", Ytest.shape)
#Ytest = to_categorical(Ytest, n_classes)
#print("Shape after one-hot encoding: ", Ytest.shape)
#print("Shape before one-hot encoding: ", Yval.shape)
#Yval = to_categorical(Yval, n_classes)
#print("Shape after one-hot encoding: ", Yval.shape)

Shape before one-hot encoding:  (920, 1)
Shape after one-hot encoding:  (920, 59)


In [8]:
type(Ytrain)

numpy.ndarray

In [9]:
Ytrain.shape

(920, 59)

In [270]:
X_train, X_test, Y_train, Y_test = train_test_split(Xtrain, 
                                                    Ytrain,
                                                    test_size=0.2,
                                                    random_state=10)

NameError: name 'X' is not defined

In [10]:
#Xtrain = np.concatenate((Xtrain, Xtest), axis=0)
#print(Xtrain.shape)
#Ytrain = np.concatenate((Ytrain, Ytest), axis=0)
#print(Ytrain.shape)

In [11]:
from sklearn.preprocessing import MinMaxScaler


num_samples, num_frames, num_keypoints = Xtrain.shape
Xtrain = Xtrain.reshape(num_samples, -1)

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
Xtrain = scaler.fit_transform(Xtrain)

# Reshape the scaled data back to the original shape
Xtrain = Xtrain.reshape(num_samples, num_frames, num_keypoints)

In [12]:
Xtrain.shape

(920, 60, 1662)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(input_data_scaled,
                                                    Y,
                                                    test_size=0.,
                                                    random_state=10)

NameError: name 'input_data_scaled' is not defined

In [13]:
print(Xtrain.shape)
#Xtest.shape

(920, 60, 1662)


In [14]:
print(Ytrain.shape)
#Ytest.shape

(920, 59)


In [15]:
Xtrain

array([[[0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.35687297, 0.60991865, 0.63696317, ..., 0.02411595,
         0.        , 1.        ],
        [0.35465615, 0.60565768, 0.66566193, ..., 0.04406686,
         0.        , 1.        ],
        [0.35232813, 0.60329661, 0.67239554, ..., 0.0836537 ,
         0.        , 1.        ]],

       [[0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 1.        , ..., 0.        ,
         0.        , 0.        ],
        ...,
        [0.33687262, 0.90758641, 0.52202365, ..., 0.02411595,
         0.        , 1.        ],
        [0.3

# Model

In [16]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam ,Adagrad, Adadelta, SGD, Lion
import os


In [17]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
tf.random.set_seed(100)

In [18]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [19]:
max_len = 60

In [18]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(LSTM(180, return_sequences=True, activation=act_function, input_shape=(60,1662)))
model.add(LSTM(90, return_sequences=True, activation=act_function))
model.add(LSTM(180, return_sequences=True, activation=act_function))
model.add(LSTM(90, return_sequences=False,activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(45, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(59, activation=act_function))
model.summary()

optimizer = SGD(learning_rate=.00001,nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)

2023-08-07 21:31:15.975071: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1037 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 130, 180)          1326960   
                                                                 
 lstm_1 (LSTM)               (None, 130, 90)           97560     
                                                                 
 lstm_2 (LSTM)               (None, 130, 180)          195120    
                                                                 
 lstm_3 (LSTM)               (None, 90)                97560     
                                                                 
 dense (Dense)               (None, 90)                8190      
                                                                 
 dense_1 (Dense)             (None, 90)                8190      
                                                                 
 dense_2 (Dense)             (None, 90)                8

2023-08-07 21:31:23.890758: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 21:31:24.039649: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0xa6a3900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 21:31:24.039679: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 21:31:24.100011: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 10s 360ms/step - loss: 8.4307 - categorical_accuracy: 0.0171 - val_loss: 5.4406 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 2s 218ms/step - loss: 7.6683 - categorical_accuracy: 0.0311 - val_loss: 5.3028 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 2s 220ms/step - loss: 7.6718 - categorical_accuracy: 0.0249 - val_loss: 5.0410 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 2s 221ms/step - loss: 8.3987 - categorical_accuracy: 0.0187 - val_loss: 5.0319 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
11/11 [==============================] - 2s 221ms/step - loss: 7.4197 - categorical_accuracy: 0.0280 - val_loss: 5.5538 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 2s 219ms/step - loss: 8.0407 - categorical_accuracy: 0.0311 - val_loss: 7.9545 - val_categorical_a

KeyboardInterrupt: 

In [18]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(GRU(64, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation=act_function))
optimizer = Adagrad(learning_rate=.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)
model.summary()

2023-08-07 21:49:05.468776: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1037 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/1000


2023-08-07 21:49:09.595786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-07 21:49:09.643909: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8864929fe0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-07 21:49:09.643936: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-07 21:49:09.647840: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-07 21:49:09.756578: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


11/11 [==============================] - 5s 178ms/step - loss: 7.8501 - categorical_accuracy: 0.0311 - val_loss: 8.1508 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 1s 82ms/step - loss: 8.1865 - categorical_accuracy: 0.0218 - val_loss: 9.0874 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 1s 82ms/step - loss: 7.9489 - categorical_accuracy: 0.0171 - val_loss: 9.2725 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 1s 82ms/step - loss: 8.4394 - categorical_accuracy: 0.0264 - val_loss: 7.0158 - val_categorical_accuracy: 0.0435
Epoch 5/1000
11/11 [==============================] - 1s 82ms/step - loss: 7.9150 - categorical_accuracy: 0.0264 - val_loss: 6.5328 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 1s 83ms/step - loss: 7.8367 - categorical_accuracy: 0.0249 - val_loss: 7.1868 - val_categorical_accuracy: 0

KeyboardInterrupt: 

In [57]:
model = Sequential()

act_function = 'LeakyReLU'
model.add(LSTM(64, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(128, return_sequences=True, activation=act_function))
model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(32, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = SGD(learning_rate=.0000005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
21/21 [==============================] - 16s 566ms/step - loss: 3.9102 - categorical_accuracy: 0.0218 - val_loss: 3.9294 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
21/21 [==============================] - 11s 541ms/step - loss: 3.9102 - categorical_accuracy: 0.0218 - val_loss: 3.9294 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
 7/21 [=========>....................] - ETA: 7s - loss: 3.9032 - categorical_accuracy: 0.0223

KeyboardInterrupt: 

In [ ]:
model = Sequential(name='Sequential') #good, overfitting

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=True, activation=act_function, input_shape=(130,1662)))
model.add(GRU(128, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,1662)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.4, batch_size = 64, callbacks=[tb_callback])
model.summary()

In [46]:
res = model.predict(Xval)

6/6 [==============================] - 1s 50ms/step


In [51]:
np.argmax(res[100])

23

In [52]:
np.argmax(Yval[100])

31

In [74]:
model = Sequential(name='Sequential') ##best

act_function = 'LeakyReLU'
model.add(GRU(16, return_sequences=False, activation=act_function, dropout=0,input_shape=(60,1662)))
#model.add(GRU(128, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(16, activation=act_function))
#model.add(Dense(64, activation=act_function))
#model.add(Dense(16, activation=act_function))
model.add(Dropout(.5))
model.add(Dense(59, activation='softmax'))
optimizer = Lion(learning_rate=.0005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.3, batch_size = 64, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
11/11 [==============================] - 3s 180ms/step - loss: 4.1087 - categorical_accuracy: 0.0171 - val_loss: 4.1858 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
11/11 [==============================] - 1s 50ms/step - loss: 4.0747 - categorical_accuracy: 0.0202 - val_loss: 4.3024 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
11/11 [==============================] - 1s 49ms/step - loss: 4.0751 - categorical_accuracy: 0.0140 - val_loss: 4.2856 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
11/11 [==============================] - 1s 49ms/step - loss: 4.0439 - categorical_accuracy: 0.0171 - val_loss: 4.3756 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
11/11 [==============================] - 1s 49ms/step - loss: 4.0622 - categorical_accuracy: 0.0155 - val_loss: 4.3673 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
11/11 [==============================] - 1s 55ms/step - loss: 4.0239 - categorical_accuracy: 0.0186 - val_loss: 4.4005 - val_catego

KeyboardInterrupt: 

In [43]:
model = Sequential(name='Sequential')

act_function = 'selu'
model.add(GRU(64, return_sequences=False, activation=act_function, dropout=.5,kernel_initializer='he_uniform',input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function))
#model.add(Dense(64, activation=act_function))
model.add(Dense(16, activation=act_function))
model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.005,global_clipnorm=10)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.4, batch_size = 64, callbacks=[tb_callback])
model.summary()

2023-08-08 17:36:17.297939: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1
2023-08-08 17:36:18.593263: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1040544960 exceeds 10% of free system memory.


Epoch 1/5000


2023-08-08 17:36:21.722239: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f7b18452100 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 17:36:21.722274: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-08 17:36:21.728081: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 17:36:21.741053: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-08 17:36:21.805903: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


19/19 [==============================] - 7s 234ms/step - loss: 4.0885 - categorical_accuracy: 0.0291 - val_loss: 4.1440 - val_categorical_accuracy: 0.0000e+00
Epoch 2/5000
19/19 [==============================] - 4s 217ms/step - loss: 4.0645 - categorical_accuracy: 0.0191 - val_loss: 4.0806 - val_categorical_accuracy: 0.0000e+00
Epoch 3/5000
19/19 [==============================] - 4s 225ms/step - loss: 3.9947 - categorical_accuracy: 0.0183 - val_loss: 4.0219 - val_categorical_accuracy: 0.0299
Epoch 4/5000
19/19 [==============================] - 4s 221ms/step - loss: 3.9759 - categorical_accuracy: 0.0183 - val_loss: 3.9075 - val_categorical_accuracy: 0.0149
Epoch 5/5000
19/19 [==============================] - 4s 224ms/step - loss: 3.9454 - categorical_accuracy: 0.0191 - val_loss: 3.9256 - val_categorical_accuracy: 0.0075
Epoch 6/5000
19/19 [==============================] - 4s 226ms/step - loss: 3.9341 - categorical_accuracy: 0.0282 - val_loss: 3.9267 - val_categorical_accuracy: 0.00

19/19 [==============================] - 5s 244ms/step - loss: 3.5412 - categorical_accuracy: 0.0822 - val_loss: 4.0214 - val_categorical_accuracy: 0.0000e+00
Epoch 50/5000
19/19 [==============================] - 5s 244ms/step - loss: 3.5468 - categorical_accuracy: 0.0831 - val_loss: 3.9519 - val_categorical_accuracy: 0.0000e+00
Epoch 51/5000
19/19 [==============================] - 5s 244ms/step - loss: 3.5162 - categorical_accuracy: 0.0872 - val_loss: 3.9390 - val_categorical_accuracy: 0.0075
Epoch 52/5000
19/19 [==============================] - 5s 246ms/step - loss: 3.5342 - categorical_accuracy: 0.0955 - val_loss: 3.8922 - val_categorical_accuracy: 0.0149
Epoch 53/5000
19/19 [==============================] - 5s 250ms/step - loss: 3.4977 - categorical_accuracy: 0.1030 - val_loss: 3.9604 - val_categorical_accuracy: 0.0000e+00
Epoch 54/5000
19/19 [==============================] - 5s 245ms/step - loss: 3.4793 - categorical_accuracy: 0.1071 - val_loss: 3.9357 - val_categorical_accur

KeyboardInterrupt: 

In [43]:
model = Sequential(name='Sequential') 

act_function = 'LeakyReLU'
initializer = 'lecun_uniform'
model.add(GRU(32, return_sequences=True, kernel_initializer=initializer, activation=act_function, dropout=0,input_shape=(130,1662)))
model.add(GRU(8, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function, kernel_initializer=initializer))
model.add(Dense(32, activation=act_function, kernel_initializer=initializer))
model.add(Dense(16, activation=act_function, kernel_initializer=initializer))
#model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.3, batch_size = 64, callbacks=[tb_callback])
model.summary()

2023-08-08 19:25:08.494048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


2023-08-08 19:25:09.673381: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 808928640 exceeds 10% of free system memory.
2023-08-08 19:25:10.394759: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 808928640 exceeds 10% of free system memory.


Epoch 1/1000


2023-08-08 19:25:13.937670: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f402c03b3b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-08 19:25:13.937718: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-08 19:25:13.943961: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-08 19:25:13.959021: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-08 19:25:14.021873: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


15/15 [==============================] - 10s 456ms/step - loss: 3.9890 - categorical_accuracy: 0.0224 - val_loss: 4.0937 - val_categorical_accuracy: 0.0025
Epoch 2/1000
15/15 [==============================] - 6s 382ms/step - loss: 3.9361 - categorical_accuracy: 0.0310 - val_loss: 4.1797 - val_categorical_accuracy: 0.0124
Epoch 3/1000
15/15 [==============================] - 6s 398ms/step - loss: 3.9642 - categorical_accuracy: 0.0331 - val_loss: 4.0007 - val_categorical_accuracy: 0.0075
Epoch 4/1000
15/15 [==============================] - 6s 414ms/step - loss: 3.9640 - categorical_accuracy: 0.0182 - val_loss: 4.0517 - val_categorical_accuracy: 0.0100
Epoch 5/1000
15/15 [==============================] - 6s 410ms/step - loss: 3.9601 - categorical_accuracy: 0.0267 - val_loss: 4.1521 - val_categorical_accuracy: 0.0274
Epoch 6/1000
15/15 [==============================] - 6s 418ms/step - loss: 3.9760 - categorical_accuracy: 0.0278 - val_loss: 4.3156 - val_categorical_accuracy: 0.0274
Epoc

KeyboardInterrupt: 

In [44]:
initializer

NameError: name 'initializer' is not defined

In [45]:
model = Sequential(name='Sequential')

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=False, activation=act_function, input_shape=(130,1662)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,1662)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dropout(.2))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

2023-08-08 10:10:09.127784: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 924736800 exceeds 10% of free system memory.
2023-08-08 10:10:09.961623: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 924736800 exceeds 10% of free system memory.


Epoch 1/1000
34/34 [==============================] - 9s 207ms/step - loss: 3.9179 - categorical_accuracy: 0.0224 - val_loss: 4.3422 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
34/34 [==============================] - 7s 201ms/step - loss: 3.8832 - categorical_accuracy: 0.0234 - val_loss: 5.0312 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
34/34 [==============================] - 7s 192ms/step - loss: 3.8310 - categorical_accuracy: 0.0215 - val_loss: 5.5114 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
34/34 [==============================] - 7s 202ms/step - loss: 3.8174 - categorical_accuracy: 0.0271 - val_loss: 5.8857 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
34/34 [==============================] - 7s 210ms/step - loss: 3.7950 - categorical_accuracy: 0.0336 - val_loss: 5.7742 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
34/34 [==============================] - 7s 212ms/step - loss: 3.7763 - categorical_accuracy: 0.0262 - val_loss: 6.2825 - val_c

KeyboardInterrupt: 

In [81]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D

# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'selu'

# Add convolutional layers
model.add(Conv1D(filters=16, kernel_size=2, activation=act_function, input_shape=(60, 1662)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.6))
model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.6))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
#model.add(SpatialDropout1D(0.5))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
#model.add(Dense(16, activation=act_function))
#model.add(Dense(16, activation=act_function))
model.add(Dropout(0.6))
model.add(Dense(59, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.00005)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 96, callbacks=[tb_callback])

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_17 (Conv1D)          (None, 59, 16)            53200     
                                                                 
 max_pooling1d_17 (MaxPooli  (None, 29, 16)            0         
 ng1D)                                                           
                                                                 
 dropout_17 (Dropout)        (None, 29, 16)            0         
                                                                 
 conv1d_18 (Conv1D)          (None, 28, 100)           3300      
                                                                 
 max_pooling1d_18 (MaxPooli  (None, 14, 100)           0         
 ng1D)                                                           
                                                                 
 dropout_18 (Dropout)        (None, 14, 100)           0

Epoch 37/1000
8/8 [==============================] - 1s 65ms/step - loss: 4.1005 - categorical_accuracy: 0.0258 - val_loss: 4.3966 - val_categorical_accuracy: 0.0000e+00
Epoch 38/1000
8/8 [==============================] - 1s 65ms/step - loss: 4.0964 - categorical_accuracy: 0.0163 - val_loss: 4.5598 - val_categorical_accuracy: 0.0000e+00
Epoch 39/1000
8/8 [==============================] - 0s 63ms/step - loss: 4.1334 - categorical_accuracy: 0.0299 - val_loss: 4.4114 - val_categorical_accuracy: 0.0000e+00
Epoch 40/1000
8/8 [==============================] - 1s 65ms/step - loss: 4.1018 - categorical_accuracy: 0.0285 - val_loss: 4.5172 - val_categorical_accuracy: 0.0000e+00
Epoch 41/1000
8/8 [==============================] - 0s 63ms/step - loss: 4.0980 - categorical_accuracy: 0.0272 - val_loss: 4.5305 - val_categorical_accuracy: 0.0000e+00
Epoch 42/1000
8/8 [==============================] - 1s 65ms/step - loss: 4.0656 - categorical_accuracy: 0.0353 - val_loss: 4.4715 - val_categorical_a

KeyboardInterrupt: 